In [0]:
%sql
CREATE OR REPLACE FUNCTION  t3_a00cd1_snd_arg02_dbws01.ext_parquet.business_hours_filter()
RETURNS BOOLEAN
RETURN hour(from_utc_timestamp(current_timestamp(), 'Australia/Melbourne')) BETWEEN 9 AND 12;

In [0]:
%sql
ALTER TABLE t3_a00cd1_snd_arg02_dbws01.ext_parquet.item
SET TAGS ('SHARINGMODE' = 'TIMERESTRICTED');

In [0]:
from databricks.sdk import WorkspaceClient
import json

w = WorkspaceClient()

securable_type = "CATALOG"
securable_full_name = "t3_a00cd1_snd_arg02_dbws01"
policy_name = "4TIMERESTRICTED"

policy_payload = {
 'on_securable_type': 'CATALOG',
 'on_securable_fullname': 't3_a00cd1_snd_arg02_dbws01',
 'name': '4TIMERESTRICTED',
 'comment': '',
 'to_principals': ['account users'],
 'except_principals': ['l205495@sandpit0x8a.onmicrosoft.com'],
 'for_securable_type': 'TABLE',
 'when_condition': 'hasTagValue("SHARINGMODE","TIMERESTRICTED")',
 'rule_type': 'RULE_TYPE_ROW_FILTER',
 'policy_type': 'POLICY_TYPE_ROW_FILTER',
 'row_filter': {'function_name': 't3_a00cd1_snd_arg02_dbws01.ext_parquet.business_hours_filter'}
 }



# DELETE the policy first if it exists
try:
    delete_endpoint = f"/api/2.1/unity-catalog/policies/{securable_type}/{securable_full_name}/{policy_name}"
    w.api_client.do("DELETE", delete_endpoint)
    print(f"Deleted existing policy: {policy_name}")
except Exception as e:
    if "does not exist" in str(e) or "RESOURCE_DOES_NOT_EXIST" in str(e) or "NotFound" in str(e):
        print(f"No existing policy to delete: {policy_name}")
    else:
        raise

# CREATE the policy fresh
print(f"Creating new policy: {policy_name}")
w.api_client.do(
    "POST",
    "/api/2.1/unity-catalog/policies/",
    data=json.dumps(policy_payload)
)

